Importing libraries.

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
from kan import KAN
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Load and preprocess of data.

In [ ]:
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    pass